In [1]:
import torch
import random

In [2]:
# 生成数据，y = Xw + b
def synthetic_data(w, b, num_examples):
    X = torch.normal(5, 3, size=(num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.1, y.shape)
    return X, y.reshape((-1, 1))

In [3]:
true_w = torch.tensor([-4.2, 3.7])
true_b = 3.4
num_examples = 1000
features, labels = synthetic_data(true_w, true_b, num_examples)

In [4]:
print('features:', features[0], '\nlabels:', labels[0])

features: tensor([7.9101, 4.1629]) 
labels: tensor([-14.3429])


In [5]:
# 定义batch size的数据迭代器
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = indices[i: min(i + batch_size, num_examples)]
        yield features[batch_indices], labels[batch_indices]


In [6]:
# 定义模型
def linreg(X, w, b):
    return torch.matmul(X, w) + b

In [7]:
# 定义损失函数
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [8]:
# 定义优化算法
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [13]:
# 初始化训练参数
w = torch.normal(3, 2, (2, 1), requires_grad=True)
b = torch.ones(1, requires_grad=True)
lr = 0.03
batch_size = 10
num_epochs = 30
net = linreg
loss = squared_loss

In [14]:
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.234301
epoch 2, loss 0.122382
epoch 3, loss 0.072943
epoch 4, loss 0.042907
epoch 5, loss 0.011952
epoch 6, loss 0.051420
epoch 7, loss 0.069662
epoch 8, loss 0.044228
epoch 9, loss 0.007932
epoch 10, loss 0.008947
epoch 11, loss 0.005748
epoch 12, loss 0.006237
epoch 13, loss 0.005816
epoch 14, loss 0.021853
epoch 15, loss 0.007122
epoch 16, loss 0.005630
epoch 17, loss 0.005979
epoch 18, loss 0.012111
epoch 19, loss 0.005205
epoch 20, loss 0.013459
epoch 21, loss 0.007901
epoch 22, loss 0.026601
epoch 23, loss 0.005268
epoch 24, loss 0.014419
epoch 25, loss 0.005272
epoch 26, loss 0.005473
epoch 27, loss 0.009716
epoch 28, loss 0.005331
epoch 29, loss 0.013143
epoch 30, loss 0.005380


In [15]:
print(f'w的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - b}')

w的估计误差：tensor([ 0.0054, -0.0046], grad_fn=<SubBackward0>)
b的估计误差：tensor([0.0023], grad_fn=<RsubBackward1>)
